# 잡음이 들어간 필기체 숫자를 복원하는 오토인코더 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

32 픽셀로 압축

In [2]:
encoding_dim = 32     #32 픽셀로 압축

함수형 API를 이용한 모델작성

In [3]:
input_img = tf.keras.layers.Input(shape=(784,))
encoded = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_img)
decoded = tf.keras.layers.Dense(784, activation='sigmoid')(encoded)
autoencoder = tf.keras.models.Model(input_img, decoded)

모델 Compile

In [4]:
autoencoder.compile(optimizer='adam', loss='mse')

MNIST 학습용 Data Set 이용

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, _), (x_test, _) = mnist.load_data()

모든 값을 0~1 사이로 정규화하고 28x28 이미지를 크기 784의 1차원 벡터로 만들기

In [6]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

넘파이 연산을 이용하여 영상에 노이즈를 추가

In [7]:
noise_factor = 0.55

original_train = x_train
original_test = x_test
noise_train = np.random.normal(0, 1, original_train.shape)
noise_test = np.random.normal(0, 1, original_test.shape)
noisy_train = original_train + noise_factor * noise_train
noisy_test = original_test + noise_factor * noise_test

AutoEncorder 학습 실행

In [ ]:
autoencoder.fit(noisy_train, original_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(noisy_test, original_test))

Noise 제거된 이미지 생성

In [ ]:
denoised_images = autoencoder.predict(noisy_test)

10개의 이미지 출력

In [ ]:
n = 10
plt.figure(figsize=(20, 6))
for i in range(1, n + 1):
  ax = plt.subplot(3, n, i)
  plt.imshow(noisy_test[i].reshape(28, 28), cmap='gray')
  plt.gray()
  
  ax = plt.subplot(3, n, i + n)
  plt.imshow(denoised_images[i].reshape(28, 28), cmap='gray')
  plt.gray()
  
  ax = plt.subplot(3, n, i + 2*n)
  plt.imshow(original_test[i].reshape(28, 28), cmap='gray')
  plt.gray()
plt.show()